## curve fitting 三种参数估计方法
问题背景：
给定N组输入$\mathbf{x}=(x_1,x_2,...,x_N)^T$以及它们对应的目标变量 $\mathbf{t}=(t_1,...,t_N)^T$，我们的目标是用一个多项式函数
$$y(x,\mathbf{w})=\sum_{j=0}^Mw_jx^j$$
去拟合这组数据中的输入输出关系，同时为新的输入作预测
（注意$\mathbf{w}$一共有M+1个参数，M称为多项式的阶）
1.MLE
假设目标变量服从一个均值为$y(x,\mathbf{w})$，方差为$\beta^{-1}$的高斯分布
$$ p(t|x,\mathbf{w},\beta)=\mathcal{N}(t|y(x,\mathbf{w}),\beta^{-1})=\sqrt{\frac{\beta}{2\pi}}\exp(-\frac{\beta}{2}(t-y(x,\mathbf{w}))^2)$$
这里参数为$\mathbf{w},\beta$
MLE是一种Frequentist方法，它认为参数是一个未知的定值，通过最大化样本的似然函数找到参数的估计。这种方法要工作，我们需要事先假定样本是独立同分布（i.i.d.)产生的。在独立同分布假设下，似然函数写为
$$ p(\mathbf{t}|\mathbf{x},\mathbf{w},\beta)=\prod_{i=1}^N \mathcal{N}(t_i|y(x_i,\mathbf{w}),\beta^{-1})$$
我们可以认为似然函数是关于参数的函数（因为数据已经给定了），同时我们取一个对数（其好处是防止最终概率值下溢）：
$$ \mathcal{L}(\mathbf{w},\beta)=\sum_{i=1}^N \bigg\{\frac{1}{2}ln\,(\beta)-\frac{1}{2}ln\,(2\pi)-\frac{\beta}{2}(t_i-y(x_i,\mathbf{w}))^2\bigg\}=\frac{N}{2}ln\,(\beta)-\frac{\beta}{2}\sum_{i=1}^N(t_i-y(x_i,\mathbf{w}))^2+const$$
我们先优化$\mathbf{w}$，此时可以将$\beta$视作参数，于是我们的目标变为：
$$\mathbf{w}_{ML}=\arg\max_{\mathbf{w}}-\sum_{i=1}^N(t_i-y(x_i,\mathbf{w}))^2=\arg\min_{\mathbf{w}}\sum_{i=1}^N(t_i-y(x_i,\mathbf{w}))^2$$
于是我们知道，求解$\mathbf{w}_{ML}$等价于最小化误差的平方和。求得$\mathbf{w}$的最大似然估计后我们带回到似然函数，并对$\beta$求导令导数为0，得到
$$ \beta_{ML}^{-1}=\frac{1}{N}\sum_{i=1}^N(t_i-y(x_i,\mathbf{w}_{ML}))^2$$
这里其实有个地方我不太明白，就是为什么可以先优化$\mathbf{w}$再代入到$\beta$的最大似然解中，还需要多看一些优化的书。
2.MAP
MLE方法的一个缺点是容易over-fitting。举个例子来说，如果样本中存在一个outlier，为了拟合这个outlier，我们需要对$\mathbf{w}_{ML}$作出很大的调整，从而失去了刻画一般数据的能力。而Bayesian方法通过给参数加先验避免了过拟合。
我们假设$\mathbf{w}$服从一个高斯分布：
$$p(\mathbf{w}|\alpha)=\mathcal{N}(\mathbf{w}|\mathbf{0},\alpha^{-1}\boldsymbol{I})=\Big(\frac{\alpha}{2\pi}\Big)^{(M+1)/2}\exp(-\frac{\alpha}{2}\mathbf{w}^T\mathbf{w})$$

$\alpha$称为超参数（hyperparameter），我们假定$\alpha$和$\beta$已知，根据贝叶斯公式有
$$p(\mathbf{w}|\mathbf{x},\mathbf{t},\alpha,\beta)\propto p(\mathbf{w}|\alpha)p(\mathbf{t}|\mathbf{x},\mathbf{w},\beta)$$
我们通过最大化后验概率$p(\mathbf{w}|\mathbf{x},\mathbf{t},\alpha,\beta)$找到$\mathbf{w}$的估计，这种方法称为MAP。$\mathbf{w}_{MAP}$由最小化如下目标函数得到：
$$ \frac{\beta}{2}\sum_{i=1}^N(t_i-y(x_i,\mathbf{w}))^2+\frac{\alpha}{2}\mathbf{w}^T\mathbf{w}$$
通过观察可以发现，MAP等价于在MLE的目标函数后加了一个L2正则项。

3.full Bayesian

关于式1.68的理解
在贝叶斯中，数据是已知的，只有参数w是未知的。因此式（1.68）中$x,\mathbf{x},\mathbf{t}$都是已知的，为了直观，我们可以把已知的都去掉，于是式1.68变为
$$p(t)=\int p(t|\mathbf{w})p(\mathbf{w}) d\mathbf{w}=\int  p(t,\mathbf{w})d\mathbf{w}$$
这就很好理解了，就是对$\mathbf{w}$做marginalization（运用概率论的乘法公式和加法公式（连续的情况下求和变为积分））
如果带上数据，推导是这样的：
$$\int p(t|x,\mathbf{w})p(\mathbf{w}|\mathbf{x},\mathbf{t})d\mathbf{w}=\int p(t,\mathbf{w}|x,\mathbf{x},\mathbf{t})d\mathbf{w}=p(t|x,\mathbf{x},\mathbf{t})$$
关于后验预测分布是高斯分布的证明，参考我的另一篇笔记。

总结一下各个方法。MLE和MAP都属于点估计，MLE在数据量少的场合下容易过拟合，而MAP通过对参数引入先验分布避免过拟合，等价于MLE的目标函数加正则化，如果假设参数是正态分布则对应L2正则（岭回归），如果假设参数服从拉普拉斯分布则对应L1正则（lasso）；full Bayesian认为点估计不准确，在MAP的基础上考虑了w的分布，通过对参数做marginalization概括所有的w，进一步避免过拟合

一些参考
1.http://ask.julyedu.com/question/150
2.PRML勘误
3.MLAPP 7.6 Bayesian linear regression
